# 추론 기반 기법과 신경망


## 통계 기반 기법의 문제점

통계 기반 기법의 문제점은 대규모 말뭉치를 다룰 때 $O(n^3)$의 비용이 든다는 점

- `통계 기반 기법` : 학습 데이터를 한꺼번에 처리 (배치 학습)
- `추론 기반 기법`: 학습 데이터의 일부를 사용하여 순차적으로 학습(미니배치 학습)

미니배치 학습은 계산량이 큰 작업을 처리할 때 효율적. 데이터를 작게 나눠 학습하기 때문에 연산 가능. 또한 병렬 계산도 가능하게 하여 학습 속도 또한 높일 수 있다.

## 추론 기반 기법 개요
```
you ? goodbye and I say hello.
```
문장에서 ?을 추론하는 작업. 모델은 이러한 추론 문제를 반복해서 풀면서 **단어의 출현 패턴을 학습**한다.

## 신경망에서의 단어 처리

신경망을 이용해 단어를 처리하기 위해서는 단어를 **고정 길이의 벡터**로 변환해야 한다.

대표적인 방법이 `원핫 표현 (one-hot encoding)`

In [5]:
# 완전연결계층에 의한 원핫 인코딩 형식의 단어 변환
# 7 X 3 계층, bias는 생략됨
import numpy as np

c = np.array([1, 0, 0, 0, 0, 0, 0]) # 입력
W = np.random.randn(7,3) # 가중치
h = np.matmul(c,W) # 중간 노드
print(h)

[-0.93613849  0.21559862  0.08711003]
